In [ ]:
import pandas as pd

# Load the two CSVs
df1 = pd.read_csv("output/dirty_fragments.csv")
df2 = pd.read_csv("output/simon/simon_dirty_fragments.csv")

# Convert relevant columns to consistent types (e.g., float for start/end)
df1["start"] = df1["start"].astype(float)
df1["end"] = df1["end"].astype(float)
df2["start"] = df2["start"].astype(float)
df2["end"] = df2["end"].astype(float)

for _, row in df1.iterrows():
    filename, start, end = row["filename"], row["start"], row["end"]

    df2_filtered = df2[
        (df2["filename"] == filename) & (df2["start"] == start) & (df2["end"] == end)
    ]

    if df2_filtered.empty:
        print(f"Not found in df2: {filename}, {start}, {end}")


KeyError: False